# AutoRegressive Integrated Moving Average
- https://www.machinelearningplus.com/time-series/arima-model-time-series-forecasting-python/
- https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/
- 3 parameters, Autoregressive (p), integration / degree of differencing (d), moving average (q)
- seasonal has additional 3 parameters for the seasonal component, and the seasonal frequency
- https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima_model.ARIMA.html
- arima(1,0,0): y(t) = c + a*y(t-1) + err(t)
- arima(1,0,1): y(t) = c + a*y(t-1) + err(t) + b*err(t-1)
- arima(0,1,0): y(t) - y(t-1) = c + err(t), y(t-1) is also By(t)